https://github.com/PanQiWei/AutoGPTQ#quantization-and-inference

In [ ]:
!pip install auto-gptq transformers

  Created wheel for auto-gptq: filename=auto_gptq-0.2.2-cp310-cp310-linux_x86_64.whl size=3631025 sha256=de939f75da3c9292bb2388922b0ba0e4a4898471c2f65836855ed5d8e4e4d770
  Stored in directory: /root/.cache/pip/wheels/bf/f6/50/bb6ab784e7824cbf190a1a8205d91e6543287718fb21d5b033
Successfully built auto-gptq


In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import re
import pandas as pd

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# Format notebook wrap behaviour
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# Clone model repo
%cd /content/drive/MyDrive/Projects/LL Models
!apt-get install git-lfs=2.13.3
!git clone https://huggingface.co/TheBloke/vicuna-13b-v1.3.0-GPTQ

In [ ]:
# Local Model - vicuna-13B
model_name_or_path = "/content/drive/MyDrive/Projects/LL Models/vicuna-13b-v1.3.0-GPTQ"
model_basename = "vicuna-13b-v1.3.0-GPTQ-4bit-128g.no-act.order"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=False,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

In [ ]:
# Define function to format prompt and tokenised inputs
def prompt_inference(prompt):

    prompt_template=f'''USER: {prompt}

    ASSISTANT:'''

    input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
    output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)

    # Prevent printing spurious transformers error when using pipeline with AutoGPTQ
    logging.set_verbosity(logging.CRITICAL)

    return tokenizer.decode(output[0])

#Labelling Datasets

In [ ]:
# Drug review dataset used as example
drug_df = pd.read_csv('/content/drive/MyDrive/Projects/Diabetes Exploration/Diabetes Dataset.csv')

In [ ]:
drug_df.shape

(161297, 7)

In [ ]:
# Random 100 samples for example
drug_df = drug_df.sample(100)

In [ ]:
drug_df = drug_df[["review", "rating"]]

In [ ]:
drug_df

,review,rating
12705,"""I have been in pain for 3 years and Oxycodone...",10
135477,"""I would definitely recommend this pill to any...",9
74465,"""I have been diagnosed at 54 years of age with...",3
58342,"""Was given Z pack for Bronchitis. Didn&#039;t ...",1
74020,"""Never had been constipated before first time,...",7
...,...,...
91406,"""I have been trying both 300mg and 150 mg. Dai...",9
46520,"""I took Saxenda at the highest dose for a tota...",1
76569,"""I have been taking this medicine for a little...",8
143978,"""Does not work at all!!!!!!!!! I started using...",1


In [ ]:
# Function that takes input dataframe, with a custom prompt to return a specific emotion for a review, with star rating context also
# Regex used to extract emotion label from output
def emotion_df_labelling(input_df):

    emo = []

    for x, y in input_df.itertuples(index=False):

        prompt = """Given the following review: {}, return one emotion only from the following list: [approval, disapproval, anxiety, fear, joy].
        Only return the emotion label and nothing else.
        Take into account the "rating" as well (where 10=good and 1=bad): {}""".format(x, y)

        input_string = prompt_inference(prompt)
        pattern = r"ASSISTANT: (.*?)</s>"
        output_emotion = re.search(pattern, input_string).group(1)

        emo.append(output_emotion)

    results_df = input_df.copy()
    results_df["Emotion"] = emo

    return results_df

In [ ]:
# Output dataframe
emotion_df_labelling(drug_df)

,review,rating,Emotion
12705,"""I have been in pain for 3 years and Oxycodone...",10,joy
135477,"""I would definitely recommend this pill to any...",9,Joy
74465,"""I have been diagnosed at 54 years of age with...",3,anxiety
58342,"""Was given Z pack for Bronchitis. Didn&#039;t ...",1,Disapproval
74020,"""Never had been constipated before first time,...",7,Disapproval
...,...,...,...
91406,"""I have been trying both 300mg and 150 mg. Dai...",9,Joy
46520,"""I took Saxenda at the highest dose for a tota...",1,Disapproval
76569,"""I have been taking this medicine for a little...",8,Joy
143978,"""Does not work at all!!!!!!!!! I started using...",1,disapproval
